计算每一个cluster内不同topic的中心性

In [1]:
import pandas as pd
import json
import numpy as np

计算发文量

In [2]:
with open("../output_data/papersCoop10.json", "r", encoding="utf-8") as f:
    jsonData = json.load(f)  # 解析 JSON 文件

In [3]:
# 10个cluster 的节点和边
nodesAll=[]
linksAll=[]
nodesToTopic={}
# 0~10
for i in range(11):
    nodesAll.append([])
    linksAll.append([])

for node in jsonData["nodes"]:
    cluster=node["cluster"]
    nodesAll[cluster].append((node["id"],node["year"]))
    nodesToTopic[node["id"]]=node["topic"]

print(nodesAll[1])

[('Global Topology of 3D Symmetric Tensor Fields', '2024'), ('Image-Based OA-Style Paper Pop-Up Design via Mixed-Integer Programming', '2023'), ('Multi-Granular Trend Detection for Time-Series Analysis', '2017'), ('Scalable Hypergraph Visualization', '2024'), ('A Simple Pipeline for Coherent Grid Maps', '2021'), ('Manga Vectorization and Manipulation with Procedural Simple Screentone', '2017'), ('Effectiveness of Area-to-Value Legends and Grid Lines in Contiguous Area Cartograms', '2024'), ('Enhancing Tai Chi Training System: Towards Group-Based and Hyper-Realistic Training Experiences', '2024'), ('ClipGen: A Deep Generative Model for Clipart Vectorization and Synthesis', '2022'), ('SPEULER: Semantics-preserving Euler Diagrams', '2022'), ('Feature Surfaces in Symmetric Tensor Fields Based on Eigenvalue Manifold', '2016'), ('From Noise Addition to Denoising: A Self-Variation Capture Network for Point Cloud Optimization', '2024'), ('Reclaiming the Horizon: Novel Visualization Designs for

In [4]:
# 验证数量
count=0
for i in range(11):
    count+=len(nodesAll[i])
print(count)

2756


In [5]:
for link in jsonData["links"]:
    cluster=link["cluster"]
    linksAll[cluster].append((link["source"],link["target"]))

print(linksAll[1])

[('Global Topology of 3D Symmetric Tensor Fields', 'Feature Surfaces in Symmetric Tensor Fields Based on Eigenvalue Manifold'), ('Global Topology of 3D Symmetric Tensor Fields', 'Robust and Fast Extraction of 3D Symmetric Tensor Field Topology'), ('Image-Based OA-Style Paper Pop-Up Design via Mixed-Integer Programming', 'Layered Scene Models from Single Hazy Images'), ('Scalable Hypergraph Visualization', 'MetroSets: Visualizing Sets as Metro Maps'), ('Scalable Hypergraph Visualization', 'Automatic Polygon Layout for Primal-Dual Visualization of Hypergraphs'), ('Scalable Hypergraph Visualization', 'A Simple Approach for Boundary Improvement of Euler Diagrams'), ('A Simple Pipeline for Coherent Grid Maps', 'Small Multiples with Gaps'), ('Effectiveness of Area-to-Value Legends and Grid Lines in Contiguous Area Cartograms', 'Task-Based Effectiveness of Interactive Contiguous Area Cartograms'), ('Effectiveness of Area-to-Value Legends and Grid Lines in Contiguous Area Cartograms', 'Evaluat

In [6]:
# 记录发文数量
# df列为topic，行为year
# 记录为json
jsonOutput = {}
jsonOutput["paperCounts"] = []
years = []
for i in range(2016, 2026):
    years.append(i)


for cluster in range(1,11):
    # 初始化
    df_years_topic = pd.DataFrame(np.zeros((11, 10)),columns=years,index=range(11))
    maxtopic=-1
    # 计算当前cluster每一年的文章数量
    for node in nodesAll[cluster]:
        year = int(node[1])
        topic = int(nodesToTopic[node[0]])
        maxtopic=max(maxtopic,topic)
        df_years_topic.loc[topic,year]+=1
    # 累积
    for year in range(2017,2026):
        for i in range(maxtopic+1):
            df_years_topic.loc[i, year] += df_years_topic.loc[i, year-1]
    # print(df_years_topic)
    # 记录到json(到最后一个整年)
    for year in range(2016,2025):
        for i in range(maxtopic+1):
            data = {"cluster": cluster, "topic":i,"year": year, "value": int(df_years_topic.loc[i,year])}
            jsonOutput["paperCounts"].append(data.copy())

# print(yearsCount)
print(jsonOutput)

{'paperCounts': [{'cluster': 1, 'topic': 0, 'year': 2016, 'value': 0}, {'cluster': 1, 'topic': 1, 'year': 2016, 'value': 0}, {'cluster': 1, 'topic': 2, 'year': 2016, 'value': 1}, {'cluster': 1, 'topic': 3, 'year': 2016, 'value': 1}, {'cluster': 1, 'topic': 4, 'year': 2016, 'value': 0}, {'cluster': 1, 'topic': 5, 'year': 2016, 'value': 1}, {'cluster': 1, 'topic': 6, 'year': 2016, 'value': 1}, {'cluster': 1, 'topic': 7, 'year': 2016, 'value': 0}, {'cluster': 1, 'topic': 0, 'year': 2017, 'value': 0}, {'cluster': 1, 'topic': 1, 'year': 2017, 'value': 2}, {'cluster': 1, 'topic': 2, 'year': 2017, 'value': 2}, {'cluster': 1, 'topic': 3, 'year': 2017, 'value': 2}, {'cluster': 1, 'topic': 4, 'year': 2017, 'value': 1}, {'cluster': 1, 'topic': 5, 'year': 2017, 'value': 1}, {'cluster': 1, 'topic': 6, 'year': 2017, 'value': 1}, {'cluster': 1, 'topic': 7, 'year': 2017, 'value': 0}, {'cluster': 1, 'topic': 0, 'year': 2018, 'value': 0}, {'cluster': 1, 'topic': 1, 'year': 2018, 'value': 4}, {'cluster':

计算接近中心性

In [7]:
import networkx as nx

In [8]:
jsonOutput["closeness"] = []
jsonOutput["betweenness"] = []

In [9]:
for cluster in range(1,11):

    # 初始化
    nodes = []
    df_years_topic_closs = pd.DataFrame(
        np.zeros((11, 10)), columns=years, index=range(11)
    )
    df_years_topic_between = pd.DataFrame(
        np.zeros((11, 10)), columns=years, index=range(11)
    )
    maxtopic=-1
    for year in range(2016,2025):
        topicSum_close = {}
        topicSum_between = {}
        topicNums={}

        # 构建图
        G = nx.Graph()

        # 添加节点
        nodes0 = [node[0] for node in nodesAll[cluster] if int(node[1]) == year]
        nodes+=nodes0
        G.add_nodes_from(nodes)

        # 添加边
        links=[link for link in linksAll[cluster] if link[0] in nodes and link[1] in nodes]
        G.add_edges_from(links)

        # 接近中心性
        closeness_centrality = nx.closeness_centrality(G)
        # 介数中心性
        betweenness_centrality = nx.betweenness_centrality(G)

        # 求和
        for node in nodes:
            topic = nodesToTopic[node]
            maxtopic = max(maxtopic, topic)
            topicSum_close[topic] = (
                topicSum_close.get(topic, 0) + closeness_centrality[node]
            )
            topicSum_between[topic] = (
                topicSum_between.get(topic, 0) + betweenness_centrality[node]
            )
            topicNums[topic] = topicNums.get(topic, 0) + 1
        # 求均值
        for t in range(maxtopic + 1):
            if(t in topicSum_close):
                topicSum_close[t] = topicSum_close[t] / topicNums[t]
            else:
                topicSum_close[t]=0
            if(t in topicSum_between):
                topicSum_between[t] = topicSum_between[t] / topicNums[t]
            else:
                topicSum_between[t] = 0

        # 记录到df
        for t in range(maxtopic+1):
            df_years_topic_closs.loc[t, year] = topicSum_close[t]
            df_years_topic_between.loc[t, year] = topicSum_between[t]
        # print(df_years_topic_closs)
        # print(topicSum_close)
    # print(df_years_topic_closs)
    # print(df_years_topic_between)
    # 记录到json
    for year in range(2016,2025):
    # for year in range(2024,2025):
        for t in range(maxtopic + 1):
            # v = float(df_years_topic_closs.loc[t, year])
            data = {
                "cluster": cluster,
                "topic": t,
                "year": year,
                "value": float(df_years_topic_closs.loc[t, year]),
            }
            jsonOutput["closeness"].append(data.copy())
            data["value"] = float(df_years_topic_between.loc[t,year])
            jsonOutput["betweenness"].append(data.copy())


print(jsonOutput["closeness"])

[{'cluster': 1, 'topic': 0, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 1, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 2, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 3, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 4, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 5, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 6, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 7, 'year': 2016, 'value': 0.0}, {'cluster': 1, 'topic': 0, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 1, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 2, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 3, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 4, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 5, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 6, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 7, 'year': 2017, 'value': 0.0}, {'cluster': 1, 'topic': 0, 'year': 2018, 'value': 0.0}, {'cluster': 1, 'topic': 1, 'year': 2018, 'value

In [10]:
with open("../output_data/centrality.json", "w", encoding="utf-8") as f:
    json.dump(jsonOutput, f, indent=4, ensure_ascii=False)